In [1]:
from nustar_pysolar import planning
from nustar_lunar_pointing.tracking import get_epoch_tle

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from datetime import datetime

In [2]:
fname = planning.download_occultation_times(outdir='../data/')
print(fname)


../data/NUSTAR.2017_129.SHADOW_ANALYSIS.txt


# Parse the TLE and find the one that's closest to the time you want to observe:

In [3]:
tlefile = '../data/NuSTAR.tle'
tstamp = '2017:137:09:15:00'
tstart = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                
mindt, line1, line2 = get_epoch_tle(tstart, tlefile)

# Find the unocculted times that you want to us:

In [4]:

tstamp = '2017:137:09:15:00'
tstart = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                
tstamp = '2017:140:13:05:00'
tend = datetime.strptime(tstamp, '%Y:%j:%H:%M:%S')                

orbits = planning.sunlight_periods(fname, tstart, tend)



# Set up the Skyfield geometry (including parallax from NuSTAR's orbit)

In [5]:
from skyfield.api import Loader, EarthSatellite
load = Loader('../data')

ts = load.timescale()
planets = load('jup310.bsp')

jupiter, earth = planets['jupiter'], planets['earth']

nustar_geo = EarthSatellite(line1, line2)
nustar = nustar_geo + earth



# Use the Skyfield interface to get the J2000 coordinates of Juptier for each orbit.

In [6]:
dt = 0.

# Using JPL Horizons web interface at 2017-05-19T01:34:40
horizon_ephem = SkyCoord(*[193.1535, -4.01689]*u.deg)

f = open('jupiter_20170517to20170520_pointing.txt', 'w')
last_ephem = None
for orbit in orbits:
    tstart = orbit[0]
    tend = orbit[1]
    print()
    on_time = (tend - tstart).total_seconds()
    
    point_time = tstart + 0.5*(tend - tstart)
    
    astro_time = Time(point_time)
    t = ts.from_astropy(astro_time)
    
    # Get the SkyField ephemeris:
    astrometric = earth.at(t).observe(jupiter)
    ra, dec, distance = astrometric.radec()
    radeg = ra.to(u.deg)
    decdeg = dec.to(u.deg)
    skyfield_ephem = SkyCoord(radeg, decdeg)
    
    
    print()
    print('Time used to aim: ', point_time)
    print('Pointing Information: ', tstart.isoformat()+' {}  {}'.format(radeg.value, decdeg.value))
    f.write(tstart.isoformat()+' {}  {}'.format(radeg.value, decdeg.value)+'\n')
    
    astrometric_parallax = nustar.at(t).observe(jupiter)
    ra, dec, distance = astrometric_parallax.radec()
    radeg = ra.to(u.deg)
    decdeg = dec.to(u.deg)
    skyfield_ephem_parallax = SkyCoord(radeg, decdeg)


    print('Parallax difference at start of orbit: ', skyfield_ephem_parallax.separation(skyfield_ephem))

    dt += on_time

    
f.close()
    
print('Total exposure: {} ks'.format(dt*1e-3))



Time used to aim:  2017-05-17 09:27:55
Pointing Information:  2017-05-17T08:56:40 193.25724338413698  -4.0535299478020965
Parallax difference at start of orbit:  0d00m01.3474s


Time used to aim:  2017-05-17 11:04:35
Pointing Information:  2017-05-17T10:33:20 193.25294336736857  -4.051999790568206
Parallax difference at start of orbit:  0d00m01.3478s


Time used to aim:  2017-05-17 12:41:15
Pointing Information:  2017-05-17T12:10:00 193.24865442609843  -4.0504744953151635
Parallax difference at start of orbit:  0d00m01.3482s


Time used to aim:  2017-05-17 14:18:00
Pointing Information:  2017-05-17T13:46:40 193.24437287127267  -4.048952750184888
Parallax difference at start of orbit:  0d00m01.3569s


Time used to aim:  2017-05-17 15:54:40
Pointing Information:  2017-05-17T15:23:20 193.2401060744517  -4.047437174497219
Parallax difference at start of orbit:  0d00m01.3574s


Time used to aim:  2017-05-17 17:31:20
Pointing Information:  2017-05-17T17:00:00 193.23585034187317  -4.0459264